Gap ups >10% in the past 6 months

In [ ]:
import yfinance as yf
import mplfinance as mpf
import pandas as pd

def calculate_sma(data, window):
    return data['Close'].rolling(window=window).mean()

def find_next_sma_cross(data, start_idx, short_window=10, long_window=20):
    data['SMA10'] = calculate_sma(data, short_window)
    data['SMA20'] = calculate_sma(data, long_window)
    
    for i in range(start_idx + 1, len(data)):
        if data['SMA10'].iloc[i] < data['SMA20'].iloc[i] and data['SMA10'].iloc[i-1] >= data['SMA20'].iloc[i-1]:
            return i
    return None

def upward(data, start_idx, threshold=0.02):
    next_cross_idx = find_next_sma_cross(data, start_idx)
    if next_cross_idx is None:
        return False
    
    current_price = data['Close'].iloc[start_idx]
    cross_price = data['Close'].iloc[next_cross_idx]
    days_between = next_cross_idx - start_idx
    daily_pct_change = (cross_price / current_price) ** (1 / days_between) - 1
    
    return daily_pct_change > threshold

def profit(data, daysAgo=10, perDayMin=0.01):
    data = data.copy()
    results = []

    for day in range(daysAgo, daysAgo + 11):
        data[f'DaysAgoClose_{day}'] = data['Close'].shift(day)
        valid_data = data.dropna(subset=[f'DaysAgoClose_{day}'])
        total = (valid_data['Close'] - valid_data[f'DaysAgoClose_{day}']) / valid_data[f'DaysAgoClose_{day}']
        per_day = ((total + 1) ** (1 / day)) - 1
        results.append(per_day >= perDayMin)

    return pd.concat(results, axis=1).any(axis=1)

def filter(data):
    upward_results = pd.Series([upward(data, idx) for idx in range(len(data))], index=data.index)
    return profit(data) & upward_results

def chart(data, ticker, target_date):
    start_date = target_date - pd.Timedelta(days=45)
    end_date = target_date + pd.Timedelta(days=45)
    window_data = data.loc[start_date:end_date]
    mpf_data = window_data[['Open', 'High', 'Low', 'Close', 'Volume']]
    past_10_days = window_data.loc[:target_date].iloc[-21:-1]
    past_10_days_high = past_10_days['High'].max()
    past_10_days_high_date = past_10_days['High'].idxmax()
    
    line = [
        (past_10_days_high_date, past_10_days_high),
        (target_date, window_data.loc[target_date, 'High'])
    ]
    
    mpf.plot(
        mpf_data,
        type='candle',
        style='charles',
        title=f'{ticker} around {target_date.date()}',
        savefig=f"{ticker}_{target_date.date()}.png",
        alines=dict(alines=[line], colors=['r'], linewidths=[2], alpha=0.7)
    )

def process(ticker):
    data = yf.download(ticker, period='max')
    data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    filter_results = filter(data)
    
    for target_date in data.index[filter_results]:
        chart(data, ticker, target_date)

tickers_df = pd.read_csv('tickers.csv', header=None)
tickers = tickers_df[0].tolist()

for ticker in tickers:
    process(ticker)